In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.8570508130000007
GPU (s):
0.17871584300007726
GPU speedup over CPU: 21x


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!tar -xf "./drive/My Drive/Datasets/fer_4_classes.tar.xz"

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import csv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from scipy import stats
import tensorflow_datasets as tfds

print(tf.version.VERSION)

2.3.0


In [ ]:
FEC_base = keras.models.load_model("./drive/My Drive/Models/FEC/FEC_base")
FEC_base.summary()

Model: "FEC_base"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
FaceNet_base (Functional)       (None, 8, 8, 896)    3406032     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 512)    458752      FaceNet_base[0][0]               
__________________________________________________________________________________________________
dense1_block1_0_bn (BatchNormal (None, 8, 8, 512)    2048        conv2d[0][0]                     
___________________________________________________________________________________________

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 160
IMG_WIDTH = 160
IMG_SIZE = 160
train_dir = "./4 classes fer/train/"
test_dir = "./4 classes fer/test/"
valid_dir = "./4 classes fer/valid/"
angry_dir = train_dir + "anger/"
happy_dir = train_dir + "happiness/"
neutral_dir = train_dir + "neutral/"
sad_dir = train_dir + "sadness/"

angry_dir_test = test_dir + "anger/"
happy_dir_test = test_dir + "happiness/"
neutral_dir_test = test_dir + "neutral/"
sad_dir_test = test_dir + "sadness/"

angry_dir_valid = valid_dir + "anger/"
happy_dir_valid = valid_dir + "happiness/"
neutral_dir_valid = valid_dir + "neutral/"
sad_dir_valid = valid_dir + "sadness/"

CLASS_NAMES = np.array(['angry', 'happy', 'neutral', 'sad'])
data_splits = np.array(["train", "test", "valid"])

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=50000):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  ds = ds.repeat()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [ ]:
fer_images = {
    "train": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "test": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "valid": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    }
}
classes = {'anger', 'happiness', 'neutral', 'sadness'}

for split in data_splits:
  for class_name in classes:
    parent = "./4 classes fer/" + split + "/" + class_name + "/"
    for filename in os.listdir(parent):
      im = np.array(Image.open(parent + filename).resize((IMG_SIZE, IMG_SIZE)), dtype=np.float16)
      im = np.expand_dims(im, -1)
      im = im / 255
      im = np.tile(im, [1, 1, 3])
      fer_images[split][class_name].append(im)

for split in data_splits:
  for class_name in classes:
    fer_images[split][class_name] = np.array(fer_images[split][class_name])

In [ ]:
fer_images["train"]["neutral"].shape

(10295, 160, 160, 3)

In [ ]:
fer_emb = {
    "train": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "test": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "valid": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    }
}

for split in data_splits:
  for class_name in classes:
    fer_emb[split][class_name] = FEC_base.predict(fer_images[split][class_name])
    fer_images[split][class_name] = None

In [ ]:
fer_emb["train"]["neutral"].shape

(10295, 16)

In [ ]:
fer_stats = {
    'anger': {
        'mean': 0,
        'std-div': 0
    },
    'happiness': {
        'mean': 0,
        'std-div': 0
    },
    'neutral': {
        'mean': 0,
        'std-div': 0
    },
    'sadness': {
        'mean': 0,
        'std-div': 0
    }
  }

In [ ]:
for class_name in classes:
  fer_stats[class_name]['mean'] = np.mean(fer_emb["train"][class_name], axis=0)
  fer_stats[class_name]['std-div'] = np.std(fer_emb["train"][class_name], axis=0)

In [ ]:
fer_stats["neutral"]

{'mean': array([ 0.31841138,  0.2099056 , -0.26889798,  0.3670699 ,  0.26859215,
         0.25925347, -0.18665399,  0.23738018,  0.13088176,  0.13340573,
        -0.17154755, -0.15907727,  0.07033927, -0.17548266, -0.31484964,
         0.22050314], dtype=float32),
 'std-div': array([0.07877191, 0.11670666, 0.08263082, 0.05964399, 0.08023044,
        0.09320825, 0.08059133, 0.05964437, 0.10181451, 0.07708932,
        0.0955204 , 0.09775028, 0.08862077, 0.14068086, 0.13649794,
        0.0845976 ], dtype=float32)}

In [ ]:
def get_zscore(x, class_n):
  return np.divide(np.subtract(x, fer_stats[class_n]["mean"]), fer_stats[class_n]["std-div"])

In [ ]:
fer_zscores = {
    "train": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "test": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "valid": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    }
}

for split in data_splits:
  for class_name in classes:
    fer_zscores[split][class_name] = np.array([np.abs(get_zscore(im, class_name)) for im in fer_emb[split][class_name]])

In [ ]:
fer_zscores["train"]["sadness"].shape

(3530, 16)

In [ ]:
fer_filters = {
    "train": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "test": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "valid": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    }
}

for split in data_splits:
  for class_name in classes:
    fer_filters[split][class_name] = (np.stack(fer_zscores[split][class_name]) < 3).all(axis=1)

In [ ]:
fer_emb_no_outliers = {
    "train": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "test": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    },
    "valid": {
      'anger': [],
      'happiness': [],
      'neutral': [],
      'sadness': []
    }
}

for split in data_splits:
  for class_name in classes:
    fer_emb_no_outliers[split][class_name] = fer_emb[split][class_name][fer_filters[split][class_name]]

In [ ]:
for split in data_splits:
  split_size = 0
  for class_name in classes:
    print(split + ", " + class_name)
    print(fer_emb_no_outliers[split][class_name].shape, fer_emb[split][class_name].shape, sep=", ")
    split_size += fer_emb_no_outliers[split][class_name].shape[0]
  if split == "train":
    TRAIN_SIZE = split_size
  elif split == "valid":
    VALID_SIZE = split_size
  else:
    TEST_SIZE = split_size
print(TRAIN_SIZE, VALID_SIZE, TEST_SIZE)

train, anger
train, sadness
train, neutral
train, happiness
test, anger
test, sadness
test, neutral
test, happiness
valid, anger
valid, sadness
valid, neutral
valid, happiness
23005 2860 2844


In [ ]:
class_to_one_hot = {
    'anger': np.array([1,0,0,0]),
    'happiness': np.array([0,1,0,0]),
    'neutral': np.array([0,0,1,0]),
    'sadness': np.array([0,0,0,0])
}

fer_ds = {
    "train": None,
    "test": None,
    "valid": None
}

for split in data_splits:
  class_arr = np.empty((0,16))
  labels = np.empty((0,4))
  for class_name in classes:
    label = np.tile(np.expand_dims(class_to_one_hot[class_name], 0), [fer_emb_no_outliers[split][class_name].shape[0], 1])
    labels = np.append(labels, label, axis=0)
    class_arr = np.append(class_arr, fer_emb_no_outliers[split][class_name], axis=0)
    print(class_arr.shape, labels.shape)
  fer_ds[split] = tf.data.Dataset.from_tensor_slices((class_arr, labels))

In [ ]:
for split in data_splits:
  fer_ds[split] = prepare_for_training(fer_ds[split])

In [ ]:
input_tensor = keras.layers.Input((16,))

x = keras.layers.Dense(512, activation="relu")(input_tensor)

x = keras.layers.Dense(4, activation='softmax')(x)

model = keras.Model(inputs=input_tensor, outputs=x, name="Emotion_Classifier")

model.summary()

Model: "Emotion_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
dense (Dense)                (None, 5, 5, 512)         655872    
_________________________________________________________________
dense_1 (Dense)              (None, 5, 5, 4)           2052      
Total params: 2,915,908
Trainable params: 2,881,796
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
steps_per_epoch = round(TRAIN_SIZE)//BATCH_SIZE
valid_steps = round(VALID_SIZE)//BATCH_SIZE
test_steps = round(TEST_SIZE)//BATCH_SIZE

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(fer_ds["test"], steps=test_steps)

89/89 [==============================] - 0s 3ms/step - loss: 1.1653 - accuracy: 0.1671


[1.1653153896331787, 0.1671348363161087]

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(fer_ds["train"],
                    callbacks=[callback],
                    steps_per_epoch=steps_per_epoch,
                    epochs=250,
                    validation_data=fer_ds["valid"],
                    validation_steps=valid_steps)
model.evaluate(fer_ds["test"], steps=test_steps)

Epoch 1/250
535/535 [==============================] - 2s 3ms/step - loss: 0.7710 - accuracy: 0.5018 - val_loss: 0.6782 - val_accuracy: 0.6239
Epoch 2/250
535/535 [==============================] - 2s 3ms/step - loss: 0.5565 - accuracy: 0.5750 - val_loss: 0.5734 - val_accuracy: 0.6654
Epoch 3/250
535/535 [==============================] - 2s 4ms/step - loss: 0.5052 - accuracy: 0.6078 - val_loss: 0.5540 - val_accuracy: 0.6977
Epoch 4/250
535/535 [==============================] - 2s 3ms/step - loss: 0.4835 - accuracy: 0.6296 - val_loss: 0.5029 - val_accuracy: 0.7142
Epoch 5/250
535/535 [==============================] - 2s 4ms/step - loss: 0.4710 - accuracy: 0.6384 - val_loss: 0.5090 - val_accuracy: 0.7138
Epoch 6/250
535/535 [==============================] - 2s 3ms/step - loss: 0.4647 - accuracy: 0.6383 - val_loss: 0.4873 - val_accuracy: 0.7047
Epoch 7/250
535/535 [==============================] - 2s 4ms/step - loss: 0.4591 - accuracy: 0.6422 - val_loss: 0.5117 - val_accuracy: 0.7075

[0.4111473560333252, 0.711727499961853]

In [ ]:
model.predict(fer_ds["valid"].take(1))

array([[4.31537718e-01, 2.77942270e-01, 2.90003777e-01, 5.16251719e-04],
       [4.29157615e-02, 1.33161604e-01, 8.23316574e-01, 6.06063404e-04],
       [5.08528687e-02, 9.44647193e-01, 4.43047518e-03, 6.94425908e-05],
       [2.21797638e-02, 6.27538189e-02, 9.14537668e-01, 5.28798555e-04],
       [1.79459617e-01, 8.10590327e-01, 9.83931217e-03, 1.10728426e-04],
       [2.65384875e-02, 3.34170014e-02, 9.39634144e-01, 4.10412118e-04],
       [6.65517375e-02, 3.16833742e-02, 9.01388764e-01, 3.76045238e-04],
       [1.94089152e-02, 4.53922786e-02, 9.34717476e-01, 4.81405790e-04],
       [3.10499430e-01, 2.51204699e-01, 4.37767506e-01, 5.28327655e-04],
       [1.80709466e-01, 9.53202471e-02, 7.23497927e-01, 4.72420012e-04],
       [6.09926023e-02, 2.47252673e-01, 6.91071212e-01, 6.83547987e-04],
       [9.17632226e-03, 9.85491157e-01, 5.26308967e-03, 6.94320479e-05],
       [5.63087046e-01, 4.19166476e-01, 1.76207777e-02, 1.25738850e-04],
       [6.03780113e-02, 1.95952296e-01, 7.42954493e

In [ ]:
FECNet_Classifier = keras.Model(inputs=FEC_base.input, outputs=model(FEC_base.output))
FECNet_Classifier.trainable = False
FECNet_Classifier.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
FaceNet_base (Functional)       (None, 8, 8, 896)    3406032     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 512)    458752      FaceNet_base[0][0]               
__________________________________________________________________________________________________
dense1_block1_0_bn (BatchNormal (None, 8, 8, 512)    2048        conv2d[0][0]                     
_______________________________________________________________________________________

In [ ]:
test_dir = "./4 classes fer/test/"
valid_dir = "./4 classes fer/valid/"

data_generator = ImageDataGenerator(rescale = 1./255.)
valid_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=valid_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))
test_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=test_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Found 2968 images belonging to 4 classes.
Found 2953 images belonging to 4 classes.


In [ ]:
num_valid = 2968
num_test = 2953
validation_steps = round(num_valid)//BATCH_SIZE
test_steps = round(num_test)//BATCH_SIZE

In [ ]:
FECNet_Classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
FECNet_Classifier.evaluate(valid_data_gen, steps=validation_steps)
FECNet_Classifier.evaluate(test_data_gen, steps=test_steps)

92/92 [==============================] - 6s 65ms/step - loss: 2.6766 - accuracy: 0.6559


[2.67659330368042, 0.655910313129425]

In [ ]:
FECNet_Classifier.save("./drive/My Drive/Models/FECNet_fer_no_outliers_20_8.h5")